In [1]:
import pandas as pd
import pickle
import json

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [16]:
def normalize_text(text):
    from bs4 import BeautifulSoup
    import re
    def strip_html_tags(text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text(separator=" ")
        return stripped_text
    text = strip_html_tags(text)
    # remove 'BULLET::::-'
    text = re.sub(r'BULLET::::-', ' ', text)
    # remove = if more than 1
    text = re.sub(r'={2,}', ' ', text)
    # emove duplicate spaces
    text = re.sub(r'  +', ' ', text)
    # remove duplicate newline characters if more than 2
    text = re.sub(r'\n+', '. ', text)

    # tokenize_text = tokenize(text)
    return text

In [17]:
df = pd.read_json('C:/Users/ngoph/Desktop/zalo_ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl', lines=True)

In [ ]:
# df = df.sample(frac=1).reset_index(drop=True)

In [18]:
df['text'] = df['text'].parallel_apply(normalize_text)

In [19]:
df.head(5)

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,Trang Chính. __NOEDITSECTION__.,2022-05-12 12:46:53+00:00,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society. Internet Society hay ISOC là...,2022-01-20 07:59:10+00:00,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt. Tiếng Việt, cũng gọi là tiếng Việt...",2022-05-29 03:42:42+00:00,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio. Ohio (viết tắt là OH, viết tắt cũ là O.)...",2022-04-17 08:15:22+00:00,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California. California (phát âm như ""Ca-li-phó...",2022-06-16 15:27:07+00:00,68738039


In [20]:
df.to_json('data_gen/wikipedia_20220620_cleaned_parsed.jsonl', orient='records', lines=True)

# Title matching

In [3]:
import transformers
from sentence_transformers import SentenceTransformer, util
import torch
transformers.logging.set_verbosity_error()

In [ ]:
model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base',device='cuda')

In [4]:
df = pd.read_json('data_gen/wikipedia_20220620_cleaned_parsed.jsonl', orient='records', lines=True)

In [ ]:
titles = df['title'].tolist()
titles[:5]

In [ ]:
# pickle.dump(titles, open('data_gen/titles.pkl', 'wb'))

In [ ]:
title_embeddings = model.encode(titles,convert_to_tensor=True,show_progress_bar=True,device='cuda')

In [ ]:
# pickle.dump(title_embeddings, open('data_gen/title_embeddings.pkl', 'wb'))

In [ ]:
# title_embeddings = pickle.load(open('data_gen/title_embeddings.pkl', 'rb'))

In [ ]:
q_raw = '''huyện Tuy An'''
query_embedding = model.encode(q_raw,convert_to_tensor=True,)
print(q_raw)
hits = util.semantic_search(query_embedding, title_embeddings, top_k=20)[0]

answers =[]
for hit in hits:
    corpus_id = hit['corpus_id']
    doc_score = hit['score']
    title = titles[corpus_id]
    print(f'{title} - {doc_score}')

# Document Retriever

In [6]:
sentences = df['text'].tolist()

In [7]:
len(sentences)

1273469

In [25]:
# pickle.dump(sentences,open('data_gen/sentences.pkl','wb'))

In [ ]:
# sentences = pickle.load(open('data_gen/sentences.pkl','rb'))

In [8]:
corpus_embeddings = model.encode(sentences,convert_to_tensor=True,show_progress_bar=True,device='cuda')

Batches:   0%|          | 0/39796 [00:00<?, ?it/s]

In [9]:
# save sentences
pickle.dump(corpus_embeddings,open('data_gen/corpus_embeddings.pkl','wb'))

In [ ]:
# corpus_embeddings = pickle.load(open('data_gen/corpus_embeddings.pkl','rb'))

In [10]:
corpus_embeddings.shape

torch.Size([1273469, 384])

In [11]:
with open('data/zac2022_train_merged_final.json',encoding='utf-8') as f:
    data=json.load(f)
    ddf = pd.json_normalize(data,'data')

In [12]:
ddf=ddf[ddf['title']!='']
ddf=ddf[ddf['category']=='FULL_ANNOTATION']

In [16]:
example = ddf.sample(1).iloc[0]
q_raw = 'Hiện nay ai là tổng bí thư nước Việt Nam'
query_embedding = model.encode(q_raw,convert_to_tensor=True,)
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=32)[0]
print(q_raw)
for hit in hits:
    corpus_id = hit['corpus_id']
    doc_score = hit['score']
    title = df.iloc[corpus_id]['title']
    print(f'{title} - {doc_score}')

Hiện nay ai là tổng bí thư nước Việt Nam
Hoàng Quốc Việt - 0.8278920650482178
Trần Vỹ (Việt Nam) - 0.8139057755470276
Nguyễn Quốc Cường - 0.8063709735870361
Nguyễn Lương Bằng - 0.800048828125
Nguyễn Quốc Dũng (nhà ngoại giao) - 0.7996822595596313
Khi đồng minh tháo chạy - 0.7978119850158691
Lê Đức Thọ (định hướng) - 0.7977200150489807
Võ Trọng Việt - 0.7966132164001465
Đệ Nhất Cộng hòa (Việt Nam Cộng hòa) - 0.7951737642288208
Nguyễn Quốc Cường (Nam Định) - 0.7941082715988159
Trần Quốc Hương - 0.7938316464424133
Nguyễn Đình Bin - 0.7923785448074341
Đinh Đồng Phụng Việt - 0.7912381291389465
Danh sách báo viết tại Việt Nam - 0.7910690307617188
Đỗ Văn Chiến - 0.7906888723373413
Nghiêm Kế Tổ - 0.7892177700996399
Vũ Ngọc Nhạ - 0.7890058159828186
Nguyễn Tân Cương - 0.7874261736869812
Nguyễn Văn Vịnh (định hướng) - 0.7862651944160461
Trần Minh Thư - 0.78569096326828
Nguyễn Bắc Son - 0.7851567268371582
Nguyễn Chánh (Bình Định) - 0.7848030924797058
Lương Cường - 0.7845540046691895
Nguyễn Văn Đượ